In [1]:
import socket
import pickle
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import padding
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [2]:
server_ip = 'localhost'
server_port = 5000
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((server_ip, server_port))
server_socket.listen(3)  # Listening to 3 clients

print(f"Server listening on {server_ip}:{server_port}")

Server listening on localhost:5000


In [3]:
server_private_key = ec.generate_private_key(ec.SECP256R1())
server_public_key = server_private_key.public_key()

In [4]:
client_connections = []
for i in range(3):
    conn, addr = server_socket.accept()
    print(f"Client {i+1} connected from {addr}")
    client_connections.append(conn)
    conn.send(server_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ))

Client 1 connected from ('127.0.0.1', 50854)
Client 2 connected from ('127.0.0.1', 50857)
Client 3 connected from ('127.0.0.1', 50862)


In [5]:
client_public_keys = []
for i in range(3):
    client_public_key_pem = client_connections[i].recv(1024)
    client_public_key = serialization.load_pem_public_key(client_public_key_pem)
    client_public_keys.append(client_public_key)

In [ ]:

global_weights = None
def build_model(input_dim):
    model = Sequential([
        Dense(16, activation='relu', input_shape=(input_dim,)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
model = build_model(input_dim=X.shape[1])  # Adjust input shape dynamically



num_rounds = 30
for round_num in range(num_rounds):
    print(f"Starting round {round_num + 1}/{num_rounds}")
    
    client_weights = []
    

    for i in range(3):
        data = pickle.dumps(global_weights)
   
        shared_key = server_private_key.exchange(ec.ECDH(), client_public_keys[i])
        derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data').derive(shared_key)
        aesgcm = AESGCM(derived_key)
        nonce = b'\x00' * 12  
        encrypted_data = aesgcm.encrypt(nonce, data, None)
        client_connections[i].send(encrypted_data)
    

    for i in range(3):
        encrypted_data = client_connections[i].recv(4096)
        shared_key = server_private_key.exchange(ec.ECDH(), client_public_keys[i])
        derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data').derive(shared_key)
        aesgcm = AESGCM(derived_key)
        data = aesgcm.decrypt(nonce, encrypted_data, None)
        local_weights = pickle.loads(data)
        client_weights.append(local_weights)
    

    new_weights = []
    for weights_list in zip(*client_weights):
        new_weights.append(np.mean(weights_list, axis=0))
    
    global_weights = new_weights
    model.set_weights(global_weights)
    
    print(f"Round {round_num + 1} complete.")



#model.save('federated_model.h5')
for conn in client_connections:
    conn.close()

print("Federated learning complete and connections closed.")


Starting round 1/30

c:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
